In [ ]:
!pip install transformers --quiet

In [ ]:
!pip install -q huggingface_hub # download in quiet mode

In [ ]:
pip install -U datasets #-U update the versions

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased",output_hidden_states = True,output_attentions = True)

inputs=tokenizer("Hello, my dog is cute",return_tensors="pt")
outputs=model(**inputs)

last_hidden_states=outputs.last_hidden_state
pooler_output=outputs.pooler_output
hidden_states=outputs.hidden_states
attentions=outputs.attentions

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [ ]:
len(attentions)

12

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased",output_hidden_states = True,output_attentions = True)

inputs=tokenizer("Hello,What are you doing?",return_tensors="pt")
outputs=model(**inputs)

last_hidden_states=outputs.last_hidden_state
pooler_output=outputs.pooler_output
hidden_states=outputs.hidden_states
attentions=outputs.attentions

In [ ]:
len(attentions)

12

Output  and  UseCase


---
last_hidden_state ----->Token classification,sequence tagging

---
pooler_output -----> Sentence classification(eg.sentimental analysis)


---
hidden_state -----> layerwise analysis,feature extraction

---
attentions -----> Vizualizing what the model focus on(attention heatmaps)

---







In [ ]:
from transformers  import AutoTokenizer,AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

model_name="textattack/bert-base-uncased-SST-2"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
text="I absolutly dont liked the movie.It was worst!"
inputs=tokenizer(text,return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  1045, 14689,  4747,  4904,  2135,  2123,  2102,  4669,  1996,
          3185,  1012,  2009,  2001,  5409,   999,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
with torch.no_grad():
  outputs=model(**inputs)
  logits=outputs.logits
  prob=F.softmax(logits,dim=1)
  predicted_class_id=logits.argmax().item()
  print(prob)
  print(predicted_class_id)

tensor([[9.9907e-01, 9.3425e-04]])
0


In [ ]:
labels=model.config.id2label
print(f"Predicted class:{labels[predicted_class_id]} (prob=[0])")

Predicted class:LABEL_0 (prob=[0])


# ***Tensorflow***

In [ ]:
!pip install transformers tensorflow --quiet

In [ ]:
import tensorflow as tf
from transformers import TFBertModel,BertTokenizer

class BertClassifier(tf.keras.Model):
  def __init__(self,model_name='bert-base-uncased',num_classes=2):
    super(BertClassifier,self).__init__()
    self.bert=TFBertModel.from_pretrained("bert-base-uncased")
    self.dropout=tf.keras.layers.Dropout(0.3)
    self.classifier=tf.keras.layers.Dense(num_classes,activation='softmax')

  def call(self,inputs,training=False):
    outputs=self.bert(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        token_type_ids=inputs['token_type_ids'],
        training=training)
    cls_output=outputs.pooler_output
    x=self.dropout(cls_output,training=training)
    x=self.classifier(x)
    return x

In [ ]:
#Load tokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
#Example sentence
text="I really loved this new iphone!"
#tokenize and conver to input format
encoded_input=tokenizer(text,return_tensors="tf",padding=True,truncation=True)
encoded_input

{'input_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=
array([[  101,  1045,  2428,  3866,  2023,  2047, 18059,   999,   102]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
#Load  custom model
model=BertClassifier(num_classes=2)

# Disable training Behaviour
model.trainable=False

#Predict
logits=model(encoded_input,training=False)
print(logits)
predicted_class=tf.math.argmax(logits,axis=-1).numpy()[0]
confidence=tf.reduce_max(logits,axis=-1).numpy()[0]

print(f"Predicted class:{predicted_class} (confidence={confidence})")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

tf.Tensor([[0.847352   0.15264799]], shape=(1, 2), dtype=float32)
Predicted class:0 (confidence=0.8473520278930664)
